In [ ]:
# Paso 3 con mapeo de cuantiles.
# Paso 2 sin mapeo de cuantiles.
# Obtiene la variable DNI (Direct Normal Irradiance)
# a partir de GHI usando el modelo DISC de NREL.
# Obtiene la variable POA (Plane of Array Irradiance)
# a partir de DNI y GHI con el modelo de Perez.
# Obtiene la variable "Cell Temperature"
# con el modelo NOCT Cell Temperature Model.
# Obtiene la variable P_mp (Producción AC del módulo fotovoltaico),
# con el modelo Simple Efficiency Model.

import numpy as np
import pandas as pd

import xarray as xr

# Inicializamos el dashboard de cómputo distribuido.
from dask.distributed import Client
c_lat = 5
c_lon = 5
Client( n_workers = 5, threads_per_worker = 5, memory_limit = "1.5GB" )

In [ ]:
# Cargamos el archivo.
n = 0
path_1 = f"../temp/WRF/WRF_miroc_1985_2014_{n}.nc"
path_2 = f"../temp/WRF/WRF_miroc_1985_2014_{n}_radiacion.nc"
ds = xr.open_dataset( path_1, chunks = {
    "south_north": c_lat, "west_east": c_lon } )

# Funciones trigonométricas.
def sin(x): return np.sin(np.radians(x))
def cos(x): return np.cos(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi

In [ ]:
# NREL DISC Model

# Huso horario.
TZ = -6

# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day Angle"] = 6.283185 * ( ds["time"].dt.dayofyear - 1 ) / 365
# Extraterrestrial radiation * reciprocal of
# the square of the earth radius vector.
ds["ETR"] = ( 1370 * ( 1.00011 + 0.034221*np.cos(ds["Day Angle"])
    + 0.00128*np.sin(ds["Day Angle"]) + 0.000719*np.cos(2*ds["Day Angle"])
    + 0.000077*np.sin(2*ds["Day Angle"]) ) )
# Declinación
ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day Angle"])
    + 0.070257*np.sin(ds["Day Angle"]) - 0.006758*np.cos(2*ds["Day Angle"])
    + 0.000907*np.sin(2*ds["Day Angle"]) - 0.002697*np.cos(3*ds["Day Angle"])
    + 0.00148*np.sin(3*ds["Day Angle"]) ) * 180/np.pi )
# Ecuación del tiempo.
ds["EQT"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day Angle"])
    - 0.032077*np.sin(ds["Day Angle"]) - 0.014615*np.cos(2*ds["Day Angle"])
    -0.040849*np.sin(2*ds["Day Angle"])) * 229.18 )
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * ( ds["time"].dt.hour - TZ + ds["EQT"]/60 )
# Ángulo horario.
ds["Hour Angle"] = ( 15 * ( ds["time"].dt.hour - 12
    - 0.5 + ds["EQT"]/60 + ((ds["lon"]-TZ*15)*4)/60 ) )
# Posiciones del analema solar.
ds["Sx"] = cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"])
ds["Sy"] = ( cos(ds["lat"])*sin(ds["Declination"])
    - sin(ds["lat"])*cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"]) )
ds["Sz"] = ( sin(ds["lat"])*sin(ds["Declination"])
    - cos(ds["lat"])*cos(ds["Declination"])*cos(ds["lon_subs"]-ds["lon"]) )
# Ángulo del cénit solar.
#ds["Zenith Angle"] = acos(
#    cos(ds["Declination"])*cos(ds["lat"])*cos(ds["Hour Angle"])
#    + sin(ds["Declination"])*sin(ds["lat"]) )
ds["Zenith Angle"] = acos(ds["Sz"])
# Ángulo acimutal solar.
ds["Azimuth Angle"] = acos( ( sin(ds["Declination"])
    - cos(ds["Zenith Angle"])*sin(ds["lat"]) )
   / ( sin(ds["Zenith Angle"])*cos(ds["lat"]) ) )
ds["Azimuth Angle"] = ds["Azimuth Angle"].where(
    ds["Hour Angle"] < 0, 360 - ds["Azimuth Angle"] )
# Masa de aire.
ds["Air Mass"] = ( 1/(cos(ds["Zenith Angle"])
    + 0.15/(93.885 - ds["Zenith Angle"])**1.253 ) * ds["Pressure"]/101325 )
ds["Air Mass"] = ds["Air Mass"].where( ds["Zenith Angle"] < 87.5, 0 )

ds = ds.drop_vars( ["Day Angle", "Declination",
    "EQT", "lon_subs", "Sx", "Sy", "Sz"] )

# Effective global horizontal transmittance.
ds["Kt"] = ds["GHI"] / (cos(ds["Zenith Angle"])*ds["ETR"])
ds["Kt"] = ds["Kt"].where( ds["Kt"] < 1, 1 ).where( ds["Air Mass"] > 0, 0 )
# Coeficientes.
ds["A_1"] = -5.743 + 21.77*ds["Kt"] - 27.49*ds["Kt"]**2 + 11.56*ds["Kt"]**3
ds["A_2"] = 0.512 - 1.56*ds["Kt"] + 2.286*ds["Kt"]**2 - 2.222*ds["Kt"]**3
ds["A"] = ds["A_1"].where( ds["Kt"] > 0.6, ds["A_2"] )
ds = ds.drop_vars( ["A_1", "A_2"] )
ds["B_1"] = 41.4 - 118.5*ds["Kt"] + 66.05*ds["Kt"]**2 + 31.9*ds["Kt"]**3
ds["B_2"] = 0.37 + 0.962*ds["Kt"]
ds["B"] = ds["B_1"].where( ds["Kt"] > 0.6, ds["B_2"] )
ds = ds.drop_vars( ["B_1", "B_2"] )
ds["C_1"] = -47.01 + 184.2*ds["Kt"] - 222*ds["Kt"]**2 + 73.81*ds["Kt"]**3
ds["C_2"] = -0.28 + 0.932*ds["Kt"] - 2.048*ds["Kt"]**2
ds["C"] = ds["C_1"].where( ds["Kt"] > 0.6, ds["C_2"] )
ds = ds.drop_vars( ["C_1", "C_2"] )
# Delta Kn.
ds["D_Kn"] = ds["A"] + ds["B"] * np.exp( ds["C"] * ds["Air Mass"] )
ds = ds.drop_vars( ["A", "B", "C"] )
# Direct beam atmospheric transmittance under clear-sky conditions.
ds["Knc"] = ( 0.866 - 0.122*ds["Air Mass"] + 0.0121*ds["Air Mass"]**2
    - 0.000653*ds["Air Mass"]**3 + 0.000014*ds["Air Mass"]**4 )
# Radiación normal directa.
ds["DNI"] = ds["ETR"] * ( ds["Knc"] - ds["D_Kn"] )
ds["DNI"] = ds["DNI"].where( ds["Kt"] > 0, 0 ).where( ds["DNI"] > 0, 0 )
    
ds = ds.drop_vars( ["ETR", "Kt", "D_Kn", "Knc"] )

In [ ]:
# Modelo de Pérez de Cielo Difuso.

azimuth_A = 180

ds["Angle of Incidence"] = acos( cos(ds["Zenith Angle"])*cos(ds["lat"])
    + sin(ds["Zenith Angle"])*sin(ds["lat"])
    *cos(ds["Azimuth Angle"]-azimuth_A) )
ds["DHI"] = ds["GHI"] - ds["DNI"] * cos(ds["Zenith Angle"])
K = 5.535e-6
ds["bins"] = ( ( (ds["DHI"]+ds["DNI"])/ds["DHI"] + K*ds["Zenith Angle"]**3 )
    / ( 1 + K*ds["Zenith Angle"]**3 ) )
ds["epsilon"] = ds["bins"   ].where( ds["bins"] < 6.200, 8 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] < 4.500, 7 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] < 2.800, 6 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] < 1.950, 5 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] < 1.500, 4 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] < 1.230, 3 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] < 1.065, 2 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] > 1.065, 1 )
Perez = pd.read_csv("Perez.csv", index_col = "bin" )
Ea = 1367
ds["Delta"] = ds["DHI"] * ds["Air Mass"] / Ea
for i in Perez.index[::-1]:
    for j in Perez.columns:
        ds[j] = ds["epsilon"].where(
            ds["epsilon"] < i, Perez.loc[i, j] )
ds["F1"] = ( ds["f11"] + ds["f12"]*ds["Delta"]
    + np.radians(ds["Zenith Angle"])*ds["f13"] )
ds["F1"] = ds["F1"].where( ds["Zenith Angle"] > 0, 0 )
ds["F2"] = ( ds["f21"] + ds["f22"]*ds["Delta"]
    + np.radians(ds["Zenith Angle"])*ds["f23"] )
ds["a"] = cos(ds["Angle of Incidence"])
ds["a"] = ds["a"].where( ds["a"] > 0, 0 )
ds["b"] = cos(ds["Angle of Incidence"])
ds["b"] = ds["b"].where( ds["b"] > 0, cos(85) )
ds["I_d"] = ( ( ds["DHI"] * ( (1-ds["F1"]) * ((1+cos(ds["lat"]))/2)
    + ds["F1"]*ds["a"]/ds["b"] + ds["F2"]*sin(ds["lat"])
    ) ).round( decimals = 2 ) )
ds["I_b"] = (ds["DNI"]*cos(ds["Angle of Incidence"])).round( decimals = 2 )
ds["POA"] = ds["I_b"] + ds["I_d"]
ds = ds.drop_vars( ["Hour Angle", "Zenith Angle", "Azimuth Angle",
    "Air Mass", "Angle of Incidence", "DHI", "epsilon", "bins",
    "Delta", "F1", "F2", "a", "b", "I_b", "I_d"] + list(Perez.columns) )

In [ ]:
# NOCT Cell Temperature Model.

T_NOCT    = 44 # °C
# Datos de Panel Canadian Solar 550 W
# Modelo: HiKu6 Mono PERC CS6W-550
I_mp      = 13.2 # A
V_mp      = 41.7  # V
A_m       = 1.134*2.278 # m^2
eff_ref   = I_mp * V_mp / (1000 * A_m)
tau_alpha = 0.9

# Ajuste de viento.
#v = 0.61 # Dos pisos.
v = 0.51 # Un piso.

# Ajuste de montaje.
T_adj = 2   + T_NOCT # Building integrated,
# greater than 3.5 in, or groud/rack mounted
#T_adj = 2  + T_NOCT # 2.5 to 3.5 in
#T_adj = 6  + T_NOCT # 1.5 to 2.5 in
#T_adj = 11 + T_NOCT # 0.5 to 1.5 in
#T_adj = 18 + T_NOCT # less than 0.5 in

ds["Cell Temperature"] = ( ds["Temperature"] + ds["POA"] / 800 * (T_adj-20)
    * (1-eff_ref/tau_alpha) * ( 9.5 / (5.7+3.8*v*ds["Wind Speed"]) ) )

In [ ]:
# Simple efficiency module model.

# Eficiencia por temperatura.
eff_T = -0.34
# Pérdidas del sistema.
eff_n = [ "Soiling", "Shading", "Snow", "Mismatch", "Wiring", "Connections",
    "Light-Induced Degradation", "Nameplate Rating", "Age", "Availability" ]
eff = np.array( [0.98, 0.97, 1, 0.98, 0.98,
    0.995, 0.985, 0.99, 1, 0.97] ).prod()
# Eficiencia del inversor.
eff_inv = 0.96
# Eficiencia del sistema.
eff_sys = eff_ref * eff_inv * eff
# DC to AC Size Ratio.
DC_AC = 1.2
# Inverter size.
inv_P = I_mp * V_mp / DC_AC

ds["P_mp"] = ( ds["POA"]*eff_sys*A_m *
    ( 1 + eff_T/100 * (ds["Cell Temperature"]-25-273.15) ) )
ds["P_mp"] = ds["P_mp"].where( ds["P_mp"] < inv_P, inv_P )
ds = ds.drop_vars( ["Cell Temperature", "POA"] )

ds